In [ ]:
import os
from pandas import DataFrame as DF

from mcbn.utils.helper import get_setup
from mcbn.utils.helper import get_directories_in_path
from mcbn.environment.constants import TEST_EVAL_PATH

In [ ]:
s = get_setup()
dataset_names = s['datasets']
all_models = s['models'] + ['{} const'.format(m) for m in s['models'] if 'MC' in m]

In [ ]:
split_count = {dn: 0 for dn in dataset_names}
all_results_df = None

for dirname in sorted(os.listdir(TEST_EVAL_PATH)):
    dataset_split_eval_path = os.path.join(TEST_EVAL_PATH, dirname)
    dataset_name = get_directories_in_path(dataset_split_eval_path)[0]
    
    split_count[dataset_name] += 1
    results_dir_path = os.path.join(dataset_split_eval_path, dataset_name)
    
    for m in all_models:
        results_path = os.path.join(results_dir_path, '{}.csv'.format(m))
        results_df = DF.from_csv(results_path)
        
        final_result_df = results_df.groupby('run_count').tail(1).reset_index(drop=True)
        final_result_df['split_seed'] = s['split_seed']
        final_result_df['dataset'] = dataset_name
        
        if 'MC' in m:
            base_model = m.replace(' const', '')
            optimal_results_path = os.path.join(results_dir_path, '{} optimum_predictions.csv'.format(base_model))
            optimal_results_df = DF.from_csv(optimal_results_path)
            
            # CRPS
            final_result_df['CRPS_opt'] = optimal_results_df['{} CRPS_opt'.format(base_model)].values
            
            # PLL
            final_result_df['PLL_opt'] = optimal_results_df['{} PLL_opt'.format(base_model)].values
        else:
            final_result_df['CRPS_opt'] = None
            final_result_df['PLL_opt'] = None
        
        if all_results_df is None:
            all_results_df = final_result_df
        else:
            all_results_df = all_results_df.append(final_result_df, ignore_index=True)
            

In [ ]:
col_order = ['dataset','model','split_seed','run_count','CRPS','PLL','RMSE','CRPS_opt','PLL_opt']
all_results_df = all_results_df[col_order]
all_results_df['model'].replace({'MCBN const': 'CUBN', 'MCDO const': 'CUDO'}, inplace=True)
all_results_df.to_csv('final_results.csv')